
Lien du site : 
https://www.leparking.be/?utm_content=cmp-true#!/voiture-occasion/%3Fid_pays%3D18%26tri%3Ddate

# Scrappe Le parking

## Import

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import pandas as pd
import urllib
import re
import requests
import csv
import time

from selenium.webdriver.common.service import Service
from selenium.webdriver.common.service import Service
from fake_useragent import UserAgent
from urllib3.exceptions import NewConnectionError
import logging
from datetime import datetime, timedelta
from selenium.common.exceptions import ElementClickInterceptedException

import pandas as pd 
import numpy as np 

import random
from selenium.webdriver.common.action_chains import ActionChains
from itertools import combinations
import time
import random
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
from fake_useragent import UserAgent

## Annex fonc

# Scrappe

##### Configurations du navigateur
ua = UserAgent()
user_agent = ua.random
options = webdriver.FirefoxOptions()
options.add_argument('--headless')
options.add_argument(f'user-agent={user_agent}')

# Initialisation du navigateur
driver = webdriver.Firefox(options=options)
base_url = "https://www.leparking.be/voiture-occasion/.html"

data_final = []

driver.get(base_url)

while True:
    # Insérez ici votre logique de scraping pour la page actuelle
    try:
        nom_p = driver.find_element(By.XPATH,  ".//div[contains(@class, 'title-n-price')]")
        nom = nom_p.get_attribute('textContent')
        # Divisez le texte en lignes
        lignes = nom.split('\n')

        # Affichez chaque modèle de voiture avec ses détails
        for i in range(0, len(lignes), 3):
            modele = lignes[i].strip()
            prix_initial = lignes[i + 1].strip()
            prix_actuel = lignes[i + 2].strip()

            print(f"Modèle : {modele}")
            print(f"Prix initial : {prix_initial}")
            print(f"Prix actuel : {prix_actuel}")
            

    except Exception as e:
        print(f"Une erreur s'est produite lors de la récupération des informations : {e}")
    try:
        date_loc_p = driver.find_element(By.XPATH,  ".//div[contains(@class, 'loc-date')]")
        date_loc = date_loc_p.get_attribute('textContent')
        print(date_loc)
        
    except Exception as e:
        print(f"Une erreur s'est produite lors de la récupération des informations : {e}")    
    try:
        info_p = driver.find_element(By.XPATH,  ".//div[contains(@class, 'bc-info clearfix bigScreen')]")
        info = info_p.get_attribute('textContent')
        print(info)
        print("\n---\n")
    except Exception as e:
        print(f"Une erreur s'est produite lors de la récupération des informations : {e}")    
    
    
    # Localisez le bouton 'btn-next' et vérifiez s'il est présent
    btn_next = driver.find_element(By.CLASS_NAME, 'btn-next') if driver.find_elements(By.CLASS_NAME, 'btn-next') else None

    if btn_next:
        # Cliquez sur le bouton 'btn-next'
        btn_next.click()

        # Attendez que la nouvelle page se charge (ajustez le temps d'attente si nécessaire)
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.TAG_NAME, 'body'))
        )
        
    else:
        # S'il n'y a plus de bouton 'btn-next', fin de boucle
        break

# Fermez le navigateur à la fin
driver.quit()

In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.options import Options
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from fake_useragent import UserAgent

ua = UserAgent()

user_agent = ua.random

options = Options()
options.add_argument('--headless')
options.add_argument(f'user-agent={user_agent}')

driver = webdriver.Firefox(options=options)
base_url = "https://www.leparking.be/voiture-occasion/.html"

data_final = []

driver.get(base_url)

while True:
    try:
        nom_elements = driver.find_elements(By.XPATH, ".//div[contains(@class, 'title-n-price')]")
        for nom_p in nom_elements:
            nom = nom_p.text
            lignes = nom.split('\n')

            for i in range(0, len(lignes), 3):
                modele = lignes[i].strip()
                prix_initial = lignes[i + 1].strip()
                prix_actuel = lignes[i + 2].strip()

                #print(f"Modèle : {modele}")
                #print(f"Prix initial : {prix_initial}")
                #print(f"Prix actuel : {prix_actuel}")

    except NoSuchElementException as e:
        print(f"Une erreur s'est produite lors de la récupération des informations (nom_p) : {e}")
        pass

    try:
        date_loc_elements = driver.find_elements(By.XPATH, ".//div[contains(@class, 'loc-date')]")
        for date_loc_p in date_loc_elements:
            date_loc = date_loc_p.get_attribute('textContent')
            #print(date_loc)

    except NoSuchElementException as e:
        print(f"Une erreur s'est produite lors de la récupération des informations (date_loc_p) : {e}")
        pass

    try:
        info_elements = driver.find_elements(By.XPATH, ".//div[contains(@class, 'bc-info clearfix bigScreen')]")
        for info_p in info_elements:
            info = info_p.get_attribute('textContent')
            #print(info)
            

    except NoSuchElementException as e:
        print(f"Une erreur s'est produite lors de la récupération des informations (info_p) : {e}")
        pass

    try:
        btn_next = driver.find_element(By.CLASS_NAME, 'btn-next') if driver.find_elements(By.CLASS_NAME, 'btn-next') else None

        if btn_next:
            btn_next.click()
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.TAG_NAME, 'body'))
            )
            print("+1")
            print("\n---\n")
        else:
            break

    except TimeoutException as e:
        print(f"TimeoutException: {e}")
        break
    except Exception as e:
        print(f"Une erreur s'est produite : {e}")

# Fermez le navigateur à la fin
driver.quit()


+1

---

+1

---

+1

---

+1

---

+1

---

+1

---

+1

---

+1

---

+1

---

+1

---

+1

---

+1

---

+1

---

+1

---

+1

---

+1

---

+1

---

+1

---

+1

---

+1

---

+1

---

+1

---

+1

---

+1

---

+1

---

+1

---

+1

---

+1

---

+1

---

+1

---

+1

---

+1

---

+1

---

+1

---

+1

---

+1

---

+1

---

+1

---

+1

---

+1

---

+1

---

+1

---

+1

---

+1

---

+1

---

+1

---

+1

---

+1

---

+1

---

+1

---

+1

---

+1

---

+1

---

+1

---

+1

---

+1

---

+1

---

+1

---

+1

---

+1

---

+1

---

+1

---

+1

---

+1

---

+1

---

+1

---

+1

---

+1

---

+1

---

+1

---

+1

---

+1

---

+1

---

+1

---

+1

---

+1

---

+1

---

+1

---

+1

---

+1

---



In [ ]:
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.options import Options
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from fake_useragent import UserAgent
import time





ua = UserAgent()

user_agent = ua.random

options = Options()
options.add_argument('--headless')
options.add_argument(f'user-agent={user_agent}')


driver = webdriver.Firefox(options=options)
base_url = "https://www.leparking.be/voiture-occasion/.html#!/voiture-occasion/%3Fid_pays%3D1%7C6%7C18"

data_final = []

driver.get(base_url)

column_mapping = {
    "Marque": "marque",
    "Modèle": "modele",
    "Version": "version",
    "Prix initial": "prix_initial",
    "Prix": "prix",
    "Date et lieu": "date_loc",
    "Info": "info"
}

while True:
    try:
        time.sleep(random.uniform(5, 10))
        nom_elements = driver.find_elements(By.XPATH, ".//div[contains(@class, 'title-n-price')]")
        for nom_p in nom_elements:
            nom = nom_p.text
            lignes = nom.split('\n')
            print(lignes)
            # Check if there are at least two lines before proceeding
            if len(lignes) >= 2:
                # Split the first line into words
                words = lignes[0].strip().split(' ')

                # Unpack the values based on the number of words
                marque = words[0] if words else ""
                modele = words[1] if len(words) > 1 else ""
                version = words[2] if len(words) > 2 else ""

                prix_initial, prix = None, None

                # Try to find Prix Initial and Prix in the lines
                for line in lignes[1:]:
                    if 'PRIX INITIAL' in line:
                        prix_initial = line.split(':')[-1].strip()
                    elif '€' in line:
                        prix = line.strip()

                data_final.append({
                    column_mapping["Marque"]: marque,
                    column_mapping["Modèle"]: modele,
                    column_mapping["Version"]: version,
                    column_mapping["Prix initial"]: prix_initial,
                    column_mapping["Prix"]: prix,
                })

    except NoSuchElementException as e:
        print(f"Une erreur s'est produite lors de la récupération des informations (nom_p) : {e}")
        pass

    try:
        time.sleep(random.uniform(10, 15))
        date_loc_elements = driver.find_elements(By.XPATH, ".//div[contains(@class, 'loc-date')]")
        for i, date_loc_p in enumerate(date_loc_elements):
            date_loc = date_loc_p.get_attribute('textContent').strip()
            # Check if the index is within the range of data_final
            if i < len(data_final):
                data_final[i][column_mapping["Date et lieu"]] = date_loc

    except NoSuchElementException as e:
        print(f"Une erreur s'est produite lors de la récupération des informations (date_loc_p) : {e}")
        pass

    try:
        time.sleep(random.uniform(10, 15))
        info_elements = driver.find_elements(By.XPATH, ".//div[contains(@class, 'bc-info clearfix bigScreen')]")
        for i, info_p in enumerate(info_elements):
            info = info_p.get_attribute('textContent').strip()
            # Check if the index is within the range of data_final
            if i < len(data_final):
                data_final[i][column_mapping["Info"]] = info

    except NoSuchElementException as e:
        print(f"Une erreur s'est produite lors de la récupération des informations (info_p) : {e}")
        pass
    
    csv_file_path = "fr4.csv"
    csv_columns = list(column_mapping.values())

    try:
        with open(csv_file_path, 'a', newline='', encoding='utf-8') as csv_file:
            csv_writer = csv.DictWriter(csv_file, fieldnames=csv_columns)
            # Write header only if the file is empty
            if csv_file.tell() == 0:
                csv_writer.writeheader()

            for data in data_final:
                csv_writer.writerow(data)
        print(f"Data written to {csv_file_path}")
    except Exception as e:
        print(f"Une erreur s'est produite lors de l'écriture du fichier CSV : {e}")
    
    try:
        btn_next = driver.find_element(By.CLASS_NAME, 'btn-next') if driver.find_elements(By.CLASS_NAME, 'btn-next') else None

        if btn_next:
            btn_next.click()
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.TAG_NAME, 'body'))
            )
            print("+1")
            print("\n---\n")
        else:
            break

    except TimeoutException as e:
        print(f"TimeoutException: {e}")
        break
    except Exception as e:
        print(f"Une erreur s'est produite : {e}")
        
    time.sleep(random.uniform(2, 6))
        

# Corrected indentation for driver.quit()
driver.quit()


['AUDI', 'Q7', 'V6 3.0QUATTRO', '34 400 €']
['RENAULT', 'MEGANE', 'E-TECHTECHNO', 'PRIX INITIAL : 35 900', '34 900 €']
['LADA', 'NIVA', '1.7TAIGA4X4', 'PRIX INITIAL : 12 990', '9 950 €']
['SKODA', 'FABIA COMBI', '1.4AMBIENTE', '3 300 €']
['MERCEDES', 'SPRINTER', '19 500 €']
['VOLKSWAGEN', 'GOLF', '1.2', 'PRIX INITIAL : 5 699', '5 000 €']
['BMW', 'SERIE 3 TOURING', '320', '10 500 €']
['VOLKSWAGEN', 'GOLF', '1.2TRENDLINE', 'PRIX INITIAL : 5 600', '5 000 €']
['PEUGEOT', '206 CC', '2 700 €']
['FORD', 'TRANSIT', '2.2LIMITED', 'PRIX INITIAL : 10 900', '4 900 €']
['FORD', 'TRANSIT', 'VAN2.2LIMITED', 'PRIX INITIAL : 10 900', '4 900 €']
['VOLKSWAGEN', 'GOLF', '1.6CONFORTLINE', 'PRIX INITIAL : 16 690', '15 900 €']
['VOLKSWAGEN', 'GOLF', '1.6CONFORTLINE', 'PRIX INITIAL : 16 900', '15 899 €']
['VOLKSWAGEN', 'PASSAT VARIANT', '2.0TRENDLINE', 'PRIX INITIAL : 16 900', '14 999 €']
['SUZUKI', 'CELERIO', '1.0', 'PRIX INITIAL : 6 900', '6 700 €']
['NISSAN', 'X-TRAIL', '1.6TEKNA4X4', 'PRIX INITIAL : 18 90

In [14]:
driver.quit()